#### Converting Zip files (okutama-action/TrainSetFrames.zip) and (okutama-action/TestSetFrames.zip)

In [35]:
# !pip install numpy

In [36]:
# import zipfile
# import os

# # Paths (using raw strings to avoid escape issues)
# zip_path = r"okutama-action/TrainSetFrames.zip"
# extract_dir = r"okutama-action/TrainSetFrames"

# # Create directory if it doesn't exist
# os.makedirs(extract_dir, exist_ok=True)

# # Extract ZIP
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# # Count number of files extracted
# num_files = sum(len(files) for _, _, files in os.walk(extract_dir))
# print(f"Frames extracted to: {extract_dir}")
# print(f"Total number of frames extracted: {num_files}")

In [37]:
# import zipfile
# import os

# # Paths (using raw strings to avoid escape issues)
# zip_path = r"okutama-action/TestSetFrames.zip"
# extract_dir = r"okutama-action/TestSetFrames"

# # Create directory if it doesn't exist
# os.makedirs(extract_dir, exist_ok=True)

# # Extract ZIP
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# # Count number of files extracted
# num_files = sum(len(files) for _, _, files in os.walk(extract_dir))
# print(f"Frames extracted to: {extract_dir}")
# print(f"Total number of frames extracted: {num_files}")

#### Visualizing a converted zip files

In [38]:
# !pip install Pillow


In [39]:
# import os
# import cv2
# import matplotlib.pyplot as plt

# # Paths
# frames_root = r"okutama-action/TrainSetFrames"
# labels_dir = r"okutama-action/TrainSetFrames/Labels/SingleActionLabels/3840x2160"

# # Original label resolution (always 4K)
# ORIG_W, ORIG_H = 3840, 2160

# # Number of sample frames to display
# max_samples = 1
# sample_count = 0

# # Define colors for actions
# ACTION_COLORS = {
#     "Handshaking": (255, 0, 0),       # Red
#     "Hugging": (0, 0, 255),           # Blue
#     "Reading": (255, 255, 0),         # Cyan
#     "Drinking": (0, 255, 255),        # Yellow
#     "Pushing/Pulling": (255, 0, 255), # Magenta
#     "Carrying": (128, 0, 128),        # Purple
#     "Calling": (0, 128, 255),         # Orange
#     "Running": (0, 255, 0),           # Green
#     "Walking": (128, 128, 0),         # Olive
#     "Lying": (128, 0, 0),             # Maroon
#     "Sitting": (0, 128, 128),         # Teal
#     "Standing": (192, 192, 192),      # Gray
#     "O": (255, 192, 203),             # Pink
#     "Okutama-Action": (0, 0, 0),      # Black
#     "None-Interaction": (128, 128, 128) # Dark Gray
# }

# DEFAULT_COLOR = (0, 255, 0)  # Bright green if action not in map

# def draw_bboxes(img, label_lines, frame_number, max_boxes_per_frame=10):
#     h, w, _ = img.shape
#     boxes_drawn = 0
#     seen_actions = set()  # keep one box per action type

#     for line in label_lines:
#         if boxes_drawn >= max_boxes_per_frame:
#             break

#         parts = line.strip().split()
#         if len(parts) < 11:
#             continue

#         # Parse values
#         xmin, ymin, xmax, ymax = map(float, parts[1:5])
#         line_frame = int(parts[5])
#         lost = int(parts[6])
#         occluded = int(parts[7])
#         label_class = parts[9].strip('"')
#         action = parts[10].strip('"') if len(parts) > 10 else ""

#         if line_frame != frame_number:
#             continue
#         if lost == 1:
#             continue

#         # one box per action per frame
#         # if action in seen_actions:
#         #     continue
#         # seen_actions.add(action)

#         # Rescale coordinates to frame
#         x1 = int(round(xmin * (w / ORIG_W)))
#         y1 = int(round(ymin * (h / ORIG_H)))
#         x2 = int(round(xmax * (w / ORIG_W)))
#         y2 = int(round(ymax * (h / ORIG_H)))

#         # Clip
#         x1, y1 = max(0, x1), max(0, y1)
#         x2, y2 = min(w - 1, x2), min(h - 1, y2)
#         if x2 <= x1 or y2 <= y1:
#             continue

#         # Pick color for action
#         color = ACTION_COLORS.get(action, DEFAULT_COLOR)

#         # Draw bbox + label
#         cv2.rectangle(img, (x1, y1), (x2, y2), color, 1)
#         cv2.putText(img, f"{label_class}-{action}", (x1, max(y1 - 10, 0)),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

#         boxes_drawn += 1

#     return img


# # Walk dataset
# for drone in sorted(os.listdir(frames_root)):
#     drone_path = os.path.join(frames_root, drone)
#     if not os.path.isdir(drone_path):
#         continue

#     for time_dir in sorted(os.listdir(drone_path)):
#         time_path = os.path.join(drone_path, time_dir)
#         if not os.path.isdir(time_path):
#             continue

#         extracted_path = os.path.join(time_path, "Extracted-Frames-1280x720")
#         if not os.path.isdir(extracted_path):
#             continue

#         for video_folder in sorted(os.listdir(extracted_path)):
#             video_path = os.path.join(extracted_path, video_folder)
#             if not os.path.isdir(video_path):
#                 continue

#             # Label file
#             label_file = os.path.join(labels_dir, f"{video_folder}.txt")
#             if not os.path.exists(label_file):
#                 print(f"No label file for {video_folder}")
#                 continue
#             with open(label_file, 'r') as lf:
#                 label_lines = lf.readlines()

#             # Sort frames
#             frame_files = sorted(
#                 [f for f in os.listdir(video_path) if f.lower().endswith(('.jpg', '.png'))],
#                 key=lambda x: int(os.path.splitext(x)[0])
#             )

#             for f in frame_files:
#                 frame_number = int(os.path.splitext(f)[0])
#                 img_path = os.path.join(video_path, f)
#                 img = cv2.imread(img_path)
#                 if img is None:
#                     continue
#                 img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#                 # Draw bboxes
#                 img = draw_bboxes(img, label_lines, frame_number)

#                 plt.figure(figsize=(16, 9))
#                 plt.imshow(img)
#                 plt.title(f"{drone}/{time_dir}/{video_folder} Frame {frame_number}")
#                 plt.axis('off')
#                 plt.show()

#                 sample_count += 1
#                 if sample_count >= max_samples:
#                     break
#             if sample_count >= max_samples:
#                 break
#         if sample_count >= max_samples:
#             break
#     if sample_count >= max_samples:
#         break

# print("Finished displaying frames with colored bboxes.")


#### Yolo-format conversion train, val and test

In [40]:
# import os
# import cv2

# # Paths
# frames_root = r"okutama-action/TestSetFrames"  # Test set frames
# labels_dir = r"okutama-action/TestSetFrames/Labels/SingleActionLabels/3840x2160"
# output_root = r"dataset-yolo"

# # Original label resolution
# ORIG_W, ORIG_H = 3840, 2160

# # Output structure
# os.makedirs(os.path.join(output_root, "images", "test"), exist_ok=True)
# os.makedirs(os.path.join(output_root, "labels", "test"), exist_ok=True)

# # Class mapping
# CLASS_NAMES = [
#     "Handshaking", "Hugging", "Reading", "Drinking", "PushingPulling",
#     "Carrying", "Calling", "Running", "Walking", "Lying",
#     "Sitting", "Standing", "None"
# ]
# CLASS2ID = {cls: i for i, cls in enumerate(CLASS_NAMES)}

# def convert_to_yolo(x1, y1, x2, y2, img_w, img_h):
#     bw = x2 - x1
#     bh = y2 - y1
#     cx = x1 + bw / 2
#     cy = y1 + bh / 2
#     return cx / img_w, cy / img_h, bw / img_w, bh / img_h

# def process_label_line(parts, frame_number, img_w, img_h):
#     if len(parts) < 11:
#         return None
#     line_frame = int(parts[5])
#     if line_frame != frame_number:
#         return None
#     x1 = int(round(float(parts[1]) * (img_w / ORIG_W)))
#     y1 = int(round(float(parts[2]) * (img_h / ORIG_H)))
#     x2 = int(round(float(parts[3]) * (img_w / ORIG_W)))
#     y2 = int(round(float(parts[4]) * (img_h / ORIG_H)))
#     x1, y1 = max(0, x1), max(0, y1)
#     x2, y2 = min(img_w - 1, x2), min(img_h - 1, y2)
#     if x2 <= x1 or y2 <= y1:
#         return None
#     label_class = parts[-2].strip('"')
#     action = parts[-1].strip('"')
#     if action not in CLASS2ID:
#         return None
#     cx, cy, bw, bh = convert_to_yolo(x1, y1, x2, y2, img_w, img_h)
#     return f"{CLASS2ID[action]} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}"

# # ---- Main loop ----
# all_samples = []

# for drone in sorted(os.listdir(frames_root)):
#     drone_path = os.path.join(frames_root, drone)
#     if not os.path.isdir(drone_path):
#         continue

#     for time_dir in sorted(os.listdir(drone_path)):
#         time_path = os.path.join(drone_path, time_dir)
#         extracted_path = os.path.join(time_path, "Extracted-Frames-1280x720")
#         if not os.path.isdir(extracted_path):
#             continue

#         for video_folder in sorted(os.listdir(extracted_path)):
#             video_path = os.path.join(extracted_path, video_folder)
#             if not os.path.isdir(video_path):
#                 continue

#             label_file = os.path.join(labels_dir, f"{video_folder}.txt")
#             if not os.path.exists(label_file):
#                 print(f"No label file for {video_folder}")
#                 continue
#             with open(label_file, "r") as lf:
#                 label_lines = lf.readlines()

#             frame_files = sorted(
#                 [f for f in os.listdir(video_path) if f.lower().endswith((".jpg", ".png"))],
#                 key=lambda x: int(os.path.splitext(x)[0])
#             )

#             for f in frame_files:
#                 frame_number = int(os.path.splitext(f)[0])
#                 img_path = os.path.join(video_path, f)
#                 img = cv2.imread(img_path)
#                 if img is None:
#                     continue
#                 img_h, img_w = img.shape[:2]

#                 yolo_lines = []
#                 for line in label_lines:
#                     parts = line.strip().split()
#                     ann = process_label_line(parts, frame_number, img_w, img_h)
#                     if ann:
#                         yolo_lines.append(ann)

#                 if not yolo_lines:
#                     continue

#                 all_samples.append((img_path, yolo_lines))

# # ---- Write test output ----
# for i, (img_path, yolo_lines) in enumerate(all_samples):
#     fname = f"test_{i:06d}.jpg"
#     out_img_path = os.path.join(output_root, "images", "test", fname)
#     out_lbl_path = os.path.join(output_root, "labels", "test", fname.replace(".jpg", ".txt"))

#     img = cv2.imread(img_path)
#     cv2.imwrite(out_img_path, img)

#     with open(out_lbl_path, "w") as f:
#         f.write("\n".join(yolo_lines))

# print("✅ Test set conversion to YOLO format finished!")


In [41]:
# import os
# import cv2
# import random

# # ------------------- Paths -------------------
# frames_root = r"okutama-action/TrainSetFrames"
# labels_dir = r"okutama-action/TrainSetFrames/Labels/SingleActionLabels/3840x2160"
# output_root = r"dataset-yolo"

# # Original label resolution
# ORIG_W, ORIG_H = 3840, 2160

# # Output directories
# splits = ["train", "val"]
# for s in splits:
#     os.makedirs(os.path.join(output_root, "images", s), exist_ok=True)
#     os.makedirs(os.path.join(output_root, "labels", s), exist_ok=True)

# # ------------------- Classes -------------------
# CLASS_NAMES = [
#     "Handshaking", "Hugging", "Reading", "Drinking", "PushingPulling",
#     "Carrying", "Calling", "Running", "Walking", "Lying",
#     "Sitting", "Standing", "None"
# ]
# CLASS2ID = {cls: i for i, cls in enumerate(CLASS_NAMES)}

# # ------------------- Functions -------------------
# def convert_to_yolo(x1, y1, x2, y2, img_w, img_h):
#     """Convert bbox to YOLO format (normalized cx, cy, w, h)."""
#     bw = x2 - x1
#     bh = y2 - y1
#     cx = x1 + bw / 2
#     cy = y1 + bh / 2
#     return cx / img_w, cy / img_h, bw / img_w, bh / img_h

# def process_label_line(parts, frame_number, img_w, img_h):
#     """Parse one annotation line and return YOLO string if valid."""
#     if len(parts) < 11:
#         return None

#     line_frame = int(parts[5])  # frame column in SingleActionLabels
#     if line_frame != frame_number:
#         return None

#     x1 = int(round(float(parts[1]) * (img_w / ORIG_W)))
#     y1 = int(round(float(parts[2]) * (img_h / ORIG_H)))
#     x2 = int(round(float(parts[3]) * (img_w / ORIG_W)))
#     y2 = int(round(float(parts[4]) * (img_h / ORIG_H)))

#     # Clip bounding box to image
#     x1, y1 = max(0, x1), max(0, y1)
#     x2, y2 = min(img_w - 1, x2), min(img_h - 1, y2)
#     if x2 <= x1 or y2 <= y1:
#         return None

#     label_class = parts[-2].strip('"')
#     action = parts[-1].strip('"')
#     if action not in CLASS2ID:
#         return None

#     # Convert to YOLO format
#     cx, cy, bw, bh = convert_to_yolo(x1, y1, x2, y2, img_w, img_h)
#     return f"{CLASS2ID[action]} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}"

# # ------------------- Main Loop -------------------
# all_samples = []

# for drone in sorted(os.listdir(frames_root)):
#     drone_path = os.path.join(frames_root, drone)
#     if not os.path.isdir(drone_path):
#         continue

#     for time_dir in sorted(os.listdir(drone_path)):
#         time_path = os.path.join(drone_path, time_dir)
#         extracted_path = os.path.join(time_path, "Extracted-Frames-1280x720")
#         if not os.path.isdir(extracted_path):
#             continue

#         for video_folder in sorted(os.listdir(extracted_path)):
#             video_path = os.path.join(extracted_path, video_folder)
#             if not os.path.isdir(video_path):
#                 continue

#             label_file = os.path.join(labels_dir, f"{video_folder}.txt")
#             if not os.path.exists(label_file):
#                 print(f"No label file for {video_folder}")
#                 continue
#             with open(label_file, "r") as lf:
#                 label_lines = lf.readlines()

#             frame_files = sorted(
#                 [f for f in os.listdir(video_path) if f.lower().endswith((".jpg", ".png"))],
#                 key=lambda x: int(os.path.splitext(x)[0])
#             )

#             for f in frame_files:
#                 frame_number = int(os.path.splitext(f)[0])
#                 img_path = os.path.join(video_path, f)
#                 img = cv2.imread(img_path)
#                 if img is None:
#                     continue
#                 img_h, img_w = img.shape[:2]

#                 yolo_lines = []
#                 for line in label_lines:
#                     parts = line.strip().split()
#                     ann = process_label_line(parts, frame_number, img_w, img_h)
#                     if ann:
#                         yolo_lines.append(ann)

#                 if not yolo_lines:
#                     continue

#                 all_samples.append((img_path, yolo_lines))

# # ------------------- Split Train / Val -------------------
# random.shuffle(all_samples)
# n = len(all_samples)
# train_split = int(0.7 * n)

# splits_idx = {
#     "train": all_samples[:train_split],
#     "val": all_samples[train_split:]
# }

# # ------------------- Write Output -------------------
# for split, samples in splits_idx.items():
#     for i, (img_path, yolo_lines) in enumerate(samples):
#         fname = f"{split}_{i:06d}.jpg"
#         out_img_path = os.path.join(output_root, "images", split, fname)
#         out_lbl_path = os.path.join(output_root, "labels", split, fname.replace(".jpg", ".txt"))

#         # Copy image
#         img = cv2.imread(img_path)
#         cv2.imwrite(out_img_path, img)

#         # Write YOLO labels
#         with open(out_lbl_path, "w") as f:
#             f.write("\n".join(yolo_lines))

# print("✅ YOLO dataset for train & val created successfully!")


In [42]:
import os

dataset_root = "dataset-yolo"
splits = ["train", "val", "test"]  # include test if you have it

for split in splits:
    img_dir = os.path.join(dataset_root, "images", split)
    lbl_dir = os.path.join(dataset_root, "labels", split)

    if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
        print(f"[!] {split} directory does not exist, skipping.")
        continue

    num_images = len([f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))])
    num_labels = len([f for f in os.listdir(lbl_dir) if f.lower().endswith(".txt")])

    print(f"{split.upper()}: Images = {num_images}, Labels = {num_labels}")


TRAIN: Images = 39835, Labels = 39835
VAL: Images = 17073, Labels = 17073
TEST: Images = 15698, Labels = 15698


#### Visulaizing yolo format conversion train/test/val

In [43]:
# import os
# import random
# import cv2
# import matplotlib.pyplot as plt

# # Paths
# yolo_root = r"dataset-yolo"
# split = "train"  # change to "val" or "test"
# img_dir = os.path.join(yolo_root, "images", split)
# lbl_dir = os.path.join(yolo_root, "labels", split)

# # Class names (same order as used in conversion)
# CLASS_NAMES = [
#     "Handshaking", "Hugging", "Reading", "Drinking", "PushingPulling",
#     "Carrying", "Calling", "Running", "Walking", "Lying",
#     "Sitting", "Standing", "None"
# ]

# # Assign random colors per class
# import numpy as np
# np.random.seed(42)
# COLORS = {cls: tuple(np.random.randint(0,255,3).tolist()) for cls in CLASS_NAMES}

# def draw_yolo_bboxes(img, lbl_path):
#     h, w, _ = img.shape
#     if not os.path.exists(lbl_path):
#         return img
    
#     with open(lbl_path, "r") as f:
#         lines = f.readlines()
    
#     for line in lines:
#         parts = line.strip().split()
#         if len(parts) != 5:
#             continue
#         cls_id = int(parts[0])
#         cx, cy, bw, bh = map(float, parts[1:])
        
#         # Convert YOLO → pixel coords
#         x1 = int((cx - bw/2) * w)
#         y1 = int((cy - bh/2) * h)
#         x2 = int((cx + bw/2) * w)
#         y2 = int((cy + bh/2) * h)
        
#         cls_name = CLASS_NAMES[cls_id]
#         color = COLORS[cls_name]
        
#         # Draw bbox
#         cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
#         cv2.putText(img, cls_name, (x1, max(y1-5,0)),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
#     return img

# # Pick random samples
# sample_images = random.sample(os.listdir(img_dir), 5)

# for img_file in sample_images:
#     img_path = os.path.join(img_dir, img_file)
#     lbl_path = os.path.join(lbl_dir, img_file.replace(".jpg", ".txt"))
    
#     img = cv2.imread(img_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     img = draw_yolo_bboxes(img, lbl_path)
    
#     plt.figure(figsize=(10,6))
#     plt.imshow(img)
#     plt.title(f"{split}/{img_file}")
#     plt.axis("off")
#     plt.show()


In [44]:
# import os
# import random
# import cv2
# import matplotlib.pyplot as plt

# # Paths
# yolo_root = r"dataset-yolo"
# split = "val"  # change to "val" or "test"
# img_dir = os.path.join(yolo_root, "images", split)
# lbl_dir = os.path.join(yolo_root, "labels", split)

# # Class names (same order as used in conversion)
# CLASS_NAMES = [
#     "Handshaking", "Hugging", "Reading", "Drinking", "PushingPulling",
#     "Carrying", "Calling", "Running", "Walking", "Lying",
#     "Sitting", "Standing", "None"
# ]

# # Assign random colors per class
# import numpy as np
# np.random.seed(42)
# COLORS = {cls: tuple(np.random.randint(0,255,3).tolist()) for cls in CLASS_NAMES}

# def draw_yolo_bboxes(img, lbl_path):
#     h, w, _ = img.shape
#     if not os.path.exists(lbl_path):
#         return img
    
#     with open(lbl_path, "r") as f:
#         lines = f.readlines()
    
#     for line in lines:
#         parts = line.strip().split()
#         if len(parts) != 5:
#             continue
#         cls_id = int(parts[0])
#         cx, cy, bw, bh = map(float, parts[1:])
        
#         # Convert YOLO → pixel coords
#         x1 = int((cx - bw/2) * w)
#         y1 = int((cy - bh/2) * h)
#         x2 = int((cx + bw/2) * w)
#         y2 = int((cy + bh/2) * h)
        
#         cls_name = CLASS_NAMES[cls_id]
#         color = COLORS[cls_name]
        
#         # Draw bbox
#         cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
#         cv2.putText(img, cls_name, (x1, max(y1-5,0)),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
#     return img

# # Pick random samples
# sample_images = random.sample(os.listdir(img_dir), 5)

# for img_file in sample_images:
#     img_path = os.path.join(img_dir, img_file)
#     lbl_path = os.path.join(lbl_dir, img_file.replace(".jpg", ".txt"))
    
#     img = cv2.imread(img_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     img = draw_yolo_bboxes(img, lbl_path)
    
#     plt.figure(figsize=(10,6))
#     plt.imshow(img)
#     plt.title(f"{split}/{img_file}")
#     plt.axis("off")
#     plt.show()


In [45]:
# import os
# import random
# import cv2
# import matplotlib.pyplot as plt

# # Paths
# yolo_root = r"dataset-yolo"
# split = "test"  # change to "val" or "test"
# img_dir = os.path.join(yolo_root, "images", split)
# lbl_dir = os.path.join(yolo_root, "labels", split)

# # Class names (same order as used in conversion)
# CLASS_NAMES = [
#     "Handshaking", "Hugging", "Reading", "Drinking", "PushingPulling",
#     "Carrying", "Calling", "Running", "Walking", "Lying",
#     "Sitting", "Standing", "None"
# ]

# # Assign random colors per class
# import numpy as np
# np.random.seed(42)
# COLORS = {cls: tuple(np.random.randint(0,255,3).tolist()) for cls in CLASS_NAMES}

# def draw_yolo_bboxes(img, lbl_path):
#     h, w, _ = img.shape
#     if not os.path.exists(lbl_path):
#         return img
    
#     with open(lbl_path, "r") as f:
#         lines = f.readlines()
    
#     for line in lines:
#         parts = line.strip().split()
#         if len(parts) != 5:
#             continue
#         cls_id = int(parts[0])
#         cx, cy, bw, bh = map(float, parts[1:])
        
#         # Convert YOLO → pixel coords
#         x1 = int((cx - bw/2) * w)
#         y1 = int((cy - bh/2) * h)
#         x2 = int((cx + bw/2) * w)
#         y2 = int((cy + bh/2) * h)
        
#         cls_name = CLASS_NAMES[cls_id]
#         color = COLORS[cls_name]
        
#         # Draw bbox
#         cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
#         cv2.putText(img, cls_name, (x1, max(y1-5,0)),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
#     return img

# # Pick random samples
# sample_images = random.sample(os.listdir(img_dir), 5)

# for img_file in sample_images:
#     img_path = os.path.join(img_dir, img_file)
#     lbl_path = os.path.join(lbl_dir, img_file.replace(".jpg", ".txt"))
    
#     img = cv2.imread(img_path)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
#     img = draw_yolo_bboxes(img, lbl_path)
    
#     plt.figure(figsize=(10,6))
#     plt.imshow(img)
#     plt.title(f"{split}/{img_file}")
#     plt.axis("off")
#     plt.show()


#### Training yolo-v8

- training and validation done in another files `yolo-train.py` and `yolo-val.py` file

In [46]:
# !pip install torch

In [47]:
# pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117


In [48]:
# import torch
# print(torch.version.cuda)          # CUDA version PyTorch is using
# print(torch.cuda.is_available())   # True if GPU is accessible


In [49]:
# import gc
# import torch
# from ultralytics import YOLO

# def train_yolo_from_scratch(
#     data_yaml: str,
#     epochs: int,
#     imgsz: int,
#     batch: int,
#     name: str,
#     model_variant: str = "yolov8n.pt"
# ):
#     # device = 'cpu'
#     device = 0 if torch.cuda.is_available() else 'cpu'
#     print(f"[+] Using device: {device}")
#     print(f"[+] Training from scratch with model: {model_variant}")

#     model = YOLO(model_variant)

#     # Clear memory
#     gc.collect()
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()

#     # Start training
#     model.train(
#         data=data_yaml,
#         epochs=epochs,
#         imgsz=imgsz,
#         batch=batch,
#         name=name,
#         project="runs/train",
#         device=device,
#         augment=True,
#         degrees=10,
#         scale=0.5,
#         flipud=0.2,
#         fliplr=0.5,
#         hsv_h=0.015,
#         hsv_s=0.7,
#         hsv_v=0.4,
#         mosaic=1.0,
#         mixup=0.2,
#         lr0=0.01,
#         lrf=0.01,
#         verbose=True,
#         patience=15
#     )

#     print(f"[+] Training complete. Results saved in 'runs/train/{name}'")
#     return model


In [50]:
# import os
# import shutil
# import torch

# # Delete 'runs' directory if it exists
# runs_dir = "runs"
# if os.path.exists(runs_dir):
#     shutil.rmtree(runs_dir)
#     print(f"Deleted existing '{runs_dir}' directory.")

# # Path to your dataset YAML
# data_yaml = "okutama.yaml"

# # Training parameters
# epochs = 1          # Number of training epochs
# imgsz = 640         # Image size (YOLO input size)
# batch = 16          # Batch size
# experiment_name = "okutama_v8_training"
# model_variant = "yolov8n.pt"  # Can also use yolov8s.pt, yolov8m.pt, etc.

# # Start training
# model = train_yolo_from_scratch(
#     data_yaml=data_yaml,
#     epochs=epochs,
#     imgsz=imgsz,
#     batch=batch,
#     name=experiment_name,
#     model_variant=model_variant
# )

# # After training, run validation
# metrics = model.val()
# print("Validation metrics:", metrics)

# # Predict on test set
# predictions = model.predict(
#     source="dataset_yolo/images/test",  # Path to test images
#     save=True,                          # Save predictions as images with bounding boxes
#     imgsz=imgsz,
#     device=0 if torch.cuda.is_available() else 'cpu'
# )

# print("Predictions saved in 'runs/detect/predict'")


#### Print Train Metrics

In [51]:
from pathlib import Path
import os
import pandas as pd

def find_best_model(base_dir='runs_yolo/'):
    best_paths = list(Path(base_dir).rglob('best.pt'))
    if not best_paths:
        raise FileNotFoundError("No 'best.pt' file found in the 'runs/' directory.")
    
    # Optionally, sort by latest modified time
    best_paths.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    
    print(f"[+] Found best.pt at: {best_paths[0]}")
    return str(best_paths[0])

In [52]:
import pandas as pd

def find_results_csv(directory):
    """Find the results.csv file in the specified directory."""
    for root, dirs, files in os.walk(directory):
        if 'results.csv' in files:
            return os.path.join(root, 'results.csv')
    return None

def load_results_csv(results_csv_path):
    """Load the results CSV into a pandas DataFrame."""
    return pd.read_csv(results_csv_path)

def calculate_total_epochs(df):
    """Calculate the total number of epochs from the DataFrame."""
    return df['epoch'].max()

def calculate_training_loss(epoch_data):
    """Calculate the total training loss from the given epoch data."""
    train_box_loss = epoch_data['train/box_loss']
    train_cls_loss = epoch_data['train/cls_loss']
    train_dfl_loss = epoch_data['train/dfl_loss']
    return train_box_loss + train_cls_loss + train_dfl_loss

def calculate_validation_loss(epoch_data):
    """Calculate the total validation loss from the given epoch data."""
    val_box_loss = epoch_data['val/box_loss']
    val_cls_loss = epoch_data['val/cls_loss']
    val_dfl_loss = epoch_data['val/dfl_loss']
    return val_box_loss + val_cls_loss + val_dfl_loss

def print_final_metrics(df):
    """Print the final metrics for the last epoch."""
    final_epoch_data = df.iloc[-1]

    # Calculate total training and validation loss
    train_loss = calculate_training_loss(final_epoch_data)
    val_loss = calculate_validation_loss(final_epoch_data)

    # Print overall metrics
    print("\n========== Final Training Metrics ==========")
    print(f"Training Loss: {train_loss:.6f}")
    print(f"Precision: {final_epoch_data['metrics/precision(B)']:.6f}")
    print(f"Recall: {final_epoch_data['metrics/recall(B)']:.6f}")
    print(f"mAP@0.5: {final_epoch_data['metrics/mAP50(B)']:.6f}")
    print(f"mAP@0.5:0.95: {final_epoch_data['metrics/mAP50-95(B)']:.6f}")

    print("\n========== Final Validation Metrics ==========")
    print(f"Validation Loss: {val_loss:.6f}")


def print_csv_metrics(directory):
    """Main function to process and print final metrics."""
    # Find the results.csv file
    results_csv_path = find_results_csv(directory)
    
    if not results_csv_path:
        print("Error: 'results.csv' file not found in the specified directory.")
        return

    print(f"Found results.csv at: {results_csv_path}")

    # Load results CSV
    df = load_results_csv(results_csv_path)

    # Get the total number of epochs
    total_epochs = calculate_total_epochs(df)
    print(f"Total number of epochs: {total_epochs}")

    print_final_metrics(df)

In [53]:
yolov8 = './runs/train/okutama_v8_training'
print_csv_metrics(yolov8)

Error: 'results.csv' file not found in the specified directory.


#### Print test metrics

In [54]:
import os

# Path to predictions folder
pred_folder = "runs/detect/predict"

# Iterate over all prediction subfolders (if any)
for root, dirs, files in os.walk(pred_folder):
    for f in files:
        if f.endswith(".txt"):
            txt_path = os.path.join(root, f)
            with open(txt_path, "r") as file:
                lines = file.readlines()
                if lines:
                    print(f"\nFile: {txt_path}")
                    for line in lines:
                        # Each line: class_id cx cy w h
                        parts = line.strip().split()
                        class_id = int(parts[0])
                        cx, cy, w, h = map(float, parts[1:])
                        print(f"Class {class_id}: cx={cx:.3f}, cy={cy:.3f}, w={w:.3f}, h={h:.3f}")
                else:
                    print(f"\nFile: {txt_path} -> No detections")
